# Defining the Fully Convolutional Neural Network

In [1]:
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
class FCNN(nn.Module):
    
    def __init__(self):
        super(FCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=30, kernel_size=5, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=30, out_channels=60, kernel_size=3, stride=1)
        self.conv2_drop = nn.Dropout2d()
        
        self.conv3 = nn.Conv2d(in_channels=60, out_channels=60, kernel_size=3, stride=2)
        self.conv3_drop = nn.Dropout2d()
        
        self.conv4 = nn.Conv2d(in_channels=60, out_channels=120, kernel_size=3, stride=1)
        
        self.conv5 = nn.Conv2d(in_channels=120, out_channels=120, kernel_size=3, stride=1)
        
        self.conv5 = nn.Conv2d(in_channels=120, out_channels=1, kernel_size=3, stride=1)
        
        
    def forward(self, data):
        x = F.relu(self.conv1(data))
        x = F.relu(self.conv2_drop(self.conv2(x)))
        x = F.relu(self.conv3_drop(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = torch.sigmoid(self.conv5(x))
        return x
        
network = FCNN()

# Loading the samples

In [3]:
POLLEN_DATA_CREATED = False
cur_dir = os.getcwd()
pollen_files = cur_dir + '/Data/PollenData/'
np_pollen_data = []     # single date will be [img,label]


if not POLLEN_DATA_CREATED:
    for folder in next(os.walk(pollen_files))[1]:
        if folder == 'Real':
            label = 1
        else:
            label = 0
        parent_path = os.path.join(pollen_files, folder)
        for file in os.listdir(parent_path):
            if '.png' in file:
                try:
                    path = os.path.join(parent_path, file)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    np_pollen_data.append([np.array(img), label])
                except Exception as e:
                    print(folder, file, str(e))

    np.random.shuffle(np_pollen_data)
    print("Data created.")


Data created.


In [4]:
# seperate the images and the labels into two lists:

images = []
labels = []

for i in range(len(np_pollen_data)):
    images.append(np_pollen_data[i][0])
    labels.append(np_pollen_data[i][1])

print(len(images))

3246


In [5]:
# Split into train and validation set:

train_x = images[:int(len(images)*0.8)]
train_y = labels[:int(len(images)*0.8)]
valid_x = images[int(len(images)*0.8):]
valid_y = labels[int(len(images)*0.8):]

In [6]:
print(images[1].shape)

(32, 32)


In [7]:
# Create Dataloaders:

trainloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.Tensor(train_x),torch.Tensor(train_y)),batch_size=10)
validloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.Tensor(valid_x),torch.Tensor(valid_y)),batch_size=100)

# Data Augmentation
and splitting the dataset into train and validation data. 

In [8]:
#transform to tensor
pollen_data = [[torch.from_numpy(i[0]).view(-1,32,32)/255.0,i[1]] for i in np_pollen_data]

#create datasets
data_amount = len(pollen_data)
train_size = int(data_amount*0.8)
val_size = int(data_amount*0.1)

pollen_training_set = pollen_data[:train_size]
pollen_validation_set = pollen_data[train_size:val_size]
pollen_testing_set = pollen_data[val_size:]

# Train the Network

First we have a look at the performance of the untrained model on our validation set. 

In [9]:
def validate(network, testloader):
    total = 0
    correct = 0
    # We do not need any gradiants here, since we do not train the network.
    # We are only interested in the predictions of the network on the testdata. 
    with torch.no_grad():
        
        for i, (inputs, labels) in enumerate(trainloader):
            outputs = network(torch.transpose(inputs[...,None],1,3)) 
            #print(labels.size(0))
            predicted = (outputs >= 0.9) # Predicted is a tensor of booleans 
            total += labels.size(0)
            predicted = predicted.view(predicted.size(0)) 
            #print(labels.size())
            #print((predicted == labels).sum().item())
            correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))
    
validate(network, validloader)

Accuracy of the network on the test images: 51 %


In [10]:
ep = 10;

#Which optimizer and criterion should we choose? Perform best?
criterion = nn.MSELoss()
optimizer = optim.SGD(network.parameters(), lr=0.001, momentum=0.9)


In [11]:
# from pytorch blitz tutorial

def train(network, trainloader, ep, criterion, optimizer, print_interval):
    for epoch in range(ep):
        
        running_loss = 0.0
    
        for i, (inputs, lables) in enumerate(trainloader):
    
            # zero the parameter gradients
            optimizer.zero_grad()
            #print(lables.shape)
            
            outputs = network(torch.transpose(inputs[...,None],1,3)).view(-1)
            #print(outputs.shape)
            loss = criterion(outputs, lables)
            loss.backward() #propagate the error back through the network
            optimizer.step() #adjust the weights of the network depending on the propagated error
    
            #that's it.
    
            #Some statistics:
            running_loss += loss.item()
    
            if i % print_interval == print_interval - 1:    # print every x mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / print_interval))
                running_loss = 0.0
    
    print('Finished Training')

train(network, trainloader, ep, criterion, optimizer, 32)

[1,    32] loss: 0.435
[1,    64] loss: 0.398
[1,    96] loss: 0.427
[1,   128] loss: 0.496
[1,   160] loss: 0.413
[1,   192] loss: 0.441
[1,   224] loss: 0.508
[1,   256] loss: 0.463
[2,    32] loss: 0.478
[2,    64] loss: 0.464
[2,    96] loss: 0.468
[2,   128] loss: 0.515
[2,   160] loss: 0.489
[2,   192] loss: 0.502
[2,   224] loss: 0.404
[2,   256] loss: 0.274
[3,    32] loss: 0.258
[3,    64] loss: 0.262
[3,    96] loss: 0.238
[3,   128] loss: 0.342
[3,   160] loss: 0.176
[3,   192] loss: 0.302
[3,   224] loss: 0.321
[3,   256] loss: 0.258
[4,    32] loss: 0.164
[4,    64] loss: 0.148
[4,    96] loss: 0.101
[4,   128] loss: 0.166
[4,   160] loss: 0.224
[4,   192] loss: 0.202
[4,   224] loss: 0.237
[4,   256] loss: 0.204
[5,    32] loss: 0.138
[5,    64] loss: 0.179
[5,    96] loss: 0.120
[5,   128] loss: 0.192
[5,   160] loss: 0.144
[5,   192] loss: 0.125
[5,   224] loss: 0.130
[5,   256] loss: 0.115
[6,    32] loss: 0.136
[6,    64] loss: 0.121
[6,    96] loss: 0.123
[6,   128] 

# Validation

The performance on the validation data after training:

In [12]:
validate(network, validloader)

Accuracy of the network on the test images: 88 %
